 I am using loan Data ( Dummy Data ) in this project. I am using Numy libirary in Python to clean and preprocess it so that its more applicable for further invistigation using machine learning models.
    
Data used in this project : https://drive.google.com/file/d/12uCRK5bLa70rbSYcogaGypbsyz7u5DMB/view?usp=sharing
<hr style="border:5px solid #108999"> </hr>

# importing Data

In [521]:
import numpy as np

In [522]:
np.set_printoptions(suppress=True)

The first step is to insert the CSV file and specify the delimiter ';'

In [523]:
loan_data=np.genfromtxt('D:\Data Processing with Numpy\\loan-data.csv', delimiter=';')
loan_data

array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [48010226.  ,         nan,    35000.  , ...,         nan,
                nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,
                nan,     4679.7 ],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,
                nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,
                nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,
                nan,      301.9 ]])

here we can see that there are alot of nan values which indicates that there are alot of non numeric values 
so the first step is to know the columns that entirely hold strings and we can do that using 'nanmean()' function

In [524]:
temp=np.nanmean(loan_data, axis=0)
temp

C:\Users\hp\AppData\Local\Temp\ipykernel_16308\3421128142.py:1: RuntimeWarning: Mean of empty slice
  temp=np.nanmean(loan_data, axis=0)


array([54015809.1922    ,               nan,    15273.46315789,
                     nan,    15311.04210526,               nan,
             16.61729479,      440.92217918,               nan,
                     nan,               nan,               nan,
                     nan,     3143.85094094])

using the next two code lines we can separate the numeric and non-numeric columns indixes in different two arrays (string_columns, numeric_columns)

In [525]:
string_columns=np.argwhere(np.isnan(temp)).squeeze()
string_columns

array([ 1,  3,  5,  8,  9, 10, 11, 12], dtype=int64)

In [526]:
numeric_columns=np.argwhere(np.isnan(temp)==False).squeeze()
numeric_columns

array([ 0,  2,  4,  6,  7, 13], dtype=int64)

now, we can import the CSV file into four different numpy arrays header numeric, header string, data numerica and data string


In [527]:
loan_data_numeric=np.genfromtxt('D:\Data Processing with Numpy\\loan-data.csv',
                                delimiter=';',
                               usecols=numeric_columns,
                               skip_header=1)
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,
            9452.96],
       [57693261.  ,    30000.  ,    30000.  ,         nan,      938.57,
            4679.7 ],
       [59432726.  ,    15000.  ,    15000.  ,         nan,      494.86,
            1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  ,         nan,         nan,
            2185.64],
       [46154151.  ,         nan,    10000.  ,       16.55,      354.3 ,
            3199.4 ],
       [66055249.  ,    10000.  ,    10000.  ,         nan,      309.97,
             301.9 ]])

In [528]:
loan_data_string=np.genfromtxt('D:\Data Processing with Numpy\\loan-data.csv',
                                delimiter=';',
                               usecols=string_columns,
                               skip_header=1,
                               dtype=str)
loan_data_string

array([['May-15', 'Current', ' 36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226',
        'CA'],
       ['', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
        'NY'],
       ['Sep-15', 'Current', ' 36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726',
        'PA'],
       ...,
       ['Jun-15', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990',
        'CA'],
       ['Apr-15', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151',
        'OH'],
       ['Dec-15', 'Current', ' 36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249',
        'IL']], dtype='<U69')

In [529]:
loan_head_string=np.genfromtxt('D:\Data Processing with Numpy\\loan-data.csv',
                                delimiter=';',
                               usecols=string_columns,
                               skip_footer=loan_data_string.shape[0],
                               dtype=str)
loan_head_string

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade',
       'verification_status', 'url', 'addr_state'], dtype='<U19')

In [530]:
loan_head_numeric=np.genfromtxt('D:\Data Processing with Numpy\\loan-data.csv',
                                delimiter=';',
                               usecols=numeric_columns,
                               skip_footer=loan_data_string.shape[0],
                               dtype=str)
loan_head_numeric


array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment',
       'total_pymnt'], dtype='<U11')

<hr style="border:5px solid #108999"> </hr> 

# String data Processing

After separating our data without the headers into 2 arrays we will start working on them one at a time let's start with the string data which consists of columns: issue_d, loan_status, term, grade, sub_grade, verification_status, url and addr_state

In [531]:
np.unique(loan_data_string[:,0])

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15',
       'Jun-15', 'Mar-15', 'May-15', 'Nov-15', 'Oct-15', 'Sep-15'],
      dtype='<U69')

### Issue_date
we checked the unique values of the first column in the string data array. as it's obvious, we can see that dates consists of days and months only also, all loans dates are in the same day but different month. so there is no use of mintioning the day
and we can also replace the month name with its numerical value (1-12) and the missing values with 0

In [532]:
loan_data_string[:,0]=np.chararray.strip(loan_data_string[:,0],'-15')
loan_data_string[:,0]

array(['May', '', 'Sep', ..., 'Jun', 'Apr', 'Dec'], dtype='<U69')

In [533]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

In [534]:
for i in range(months.shape[0]):
    loan_data_string[:,0]=np.where(loan_data_string[:,0]==months[i],
                                  i,
                                  loan_data_string[:,0])
np.unique(loan_data_string[:,0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'],
      dtype='<U69')

### loan_status
we checked the unique values of the second column 'Loan_statuts' in the string data array. according to the business model we should catigorize these values into two groups (good '1', bad '0')

ps: the Empty values ' ' means there were missing values in the CSV file so we assign them according to the worest case scenario

In [535]:
np.unique(loan_data_string[:,1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid',
       'In Grace Period', 'Issued', 'Late (16-30 days)',
       'Late (31-120 days)'], dtype='<U69')

In [536]:
status_bad = np.array(['','Charged Off','Default','Late (31-120 days)'])

In [537]:
loan_data_string[:,1]= np.where(np.isin(loan_data_string[:,1], status_bad),0,1)
np.unique(loan_data_string[:,1])

array(['0', '1'], dtype='<U69')

### Term
lets move to the third column 'term' which contains three unique values ('', '36 months','60 months').
we will replace '36 months' with '36' , '60 months' with '60' and ' ' with '36' as it's the worest case scenario

In [538]:
np.unique(loan_data_string[:,2])

array(['', ' 36 months', ' 60 months'], dtype='<U69')

In [539]:
loan_data_string[:,2]=np.chararray.strip(loan_data_string[:,2], ' months')
np.unique(loan_data_string[:,2])

array(['', '36', '60'], dtype='<U69')

In [540]:
loan_data_string[:,2]=np.where(loan_data_string[:,2]=='',
                              60,
                              loan_data_string[:,2])
np.unique(loan_data_string[:,2])

array(['36', '60'], dtype='<U69')

### grade and sub_grade
in the next two columns (grade column, Sub-grade column),<br>
the unique values for the grade column are (' ', 'A', 'B', 'C', 'D', 'E', 'F', 'G').<br>
and for the sub grade are the same like the grade but a numeric value from 1-5 is added after each grade.<br> so we can<br>
1- Replace the missing values of sub-grade with the the least level of grade if found for example if the grade is 'B' so the missing sub grade is 'B5'. <br>2- delete the grade column as all its info is duplicated in sub-grade.<br>3- replace the missing subgrades with a newly added value 'H1'<br>4- replace every level of sub-grade with a numeric value

In [541]:
np.unique(loan_data_string[:,4], return_counts=True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5',
        'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1',
        'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2',
        'G3', 'G4', 'G5'], dtype='<U69'),
 array([514, 285, 278, 239, 323, 502, 509, 517, 530, 553, 494, 629, 567,
        586, 564, 423, 391, 267, 250, 255, 223, 235, 162, 171, 139, 114,
         94,  52,  34,  43,  16,  19,  10,   3,   7,   2], dtype=int64))

In [542]:
# 1- Replace the missing values of sub-grade with the the least level of grade if found 
#    for example if the grade is 'B' so the missing sub grade is 'B5'.

for i in range(loan_data_string.shape[0]):
    loan_data_string[i,4]=np.where((loan_data_string[i,4]=='')&(loan_data_string[i,3]!=''),
                                  loan_data_string[i,3]+'5',
                                  loan_data_string[i,4])
    
np.unique(loan_data_string[:,4], return_counts=True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5',
        'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1',
        'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2',
        'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567,
        586, 564, 577, 391, 267, 250, 255, 288, 235, 162, 171, 139, 160,
         94,  52,  34,  43,  24,  19,  10,   3,   7,   5], dtype=int64))

In [543]:
# 2- delete the grade column as all its info is duplicated in sub-grade.

loan_data_string=np.delete(loan_data_string, 3, axis=1)
loan_data_string[:,3]
loan_head_string=np.delete(loan_head_string, 3)
loan_head_string[3]

'sub_grade'

In [544]:
# 3- replace the missing subgrades with a newly added value 'H1'

loan_data_string[:,3]=np.where(loan_data_string[:,3]=='',
                              'H1',
                              loan_data_string[:,3])

np.unique(loan_data_string[:,3], return_counts=True)

(array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1',
        'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2',
        'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3',
        'G4', 'G5', 'H1'], dtype='<U69'),
 array([285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586,
        564, 577, 391, 267, 250, 255, 288, 235, 162, 171, 139, 160,  94,
         52,  34,  43,  24,  19,  10,   3,   7,   5,   9], dtype=int64))

In [545]:
# 4- replace every level of sub-grade with a numeric value

keys= list(np.unique(loan_data_string[:,3]))
values= list(range(1,np.unique(loan_data_string[:,3]).shape[0]+1))
temp_dict=dict(zip(keys, values))
temp_dict

{'A1': 1,
 'A2': 2,
 'A3': 3,
 'A4': 4,
 'A5': 5,
 'B1': 6,
 'B2': 7,
 'B3': 8,
 'B4': 9,
 'B5': 10,
 'C1': 11,
 'C2': 12,
 'C3': 13,
 'C4': 14,
 'C5': 15,
 'D1': 16,
 'D2': 17,
 'D3': 18,
 'D4': 19,
 'D5': 20,
 'E1': 21,
 'E2': 22,
 'E3': 23,
 'E4': 24,
 'E5': 25,
 'F1': 26,
 'F2': 27,
 'F3': 28,
 'F4': 29,
 'F5': 30,
 'G1': 31,
 'G2': 32,
 'G3': 33,
 'G4': 34,
 'G5': 35,
 'H1': 36}

In [546]:
for i in temp_dict.keys():
    loan_data_string[:,3]=np.where(loan_data_string[:,3]==i,
                                  temp_dict[i],
                                  loan_data_string[:,3])
np.unique(loan_data_string[:,3])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
       '3', '30', '31', '32', '33', '34', '35', '36', '4', '5', '6', '7',
       '8', '9'], dtype='<U69')

### Verification Status

for column 'verification_status', the unique values in this column are (' ',Not Verified, Sourece Verified, Verified)
so we can process these data in the following steps:<br>1- replace values (' ',Not Verified) with 0<br>2- replace (Sourece Verified, Verified) with 1

In [547]:
np.unique(loan_data_string[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

In [548]:
loan_data_string[:,4]=np.where((loan_data_string[:,4]=='')|(loan_data_string[:,4]=='Not Verified'),
                              0,1)
np.unique(loan_data_string[:,4])

array(['0', '1'], dtype='<U69')

### URL
for the column 'URL' we can see that all the values like this 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226' with except change in the id part at the end of the URL also we can see that this id at the end is the same as the id column in the numeric data<br>Then there is no need for this column as it doesn't add value 

In [549]:
loan_data_string[:5,5]

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=53222800',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57803010'],
      dtype='<U69')

In [550]:
loan_data_numeric[:5,0]

array([48010226., 57693261., 59432726., 53222800., 57803010.])

In [551]:
# the first 5 rows from URL column 
loan_data_string=np.delete(loan_data_string, 5, axis=1)
loan_head_string=np.delete(loan_head_string, 5)
print(loan_head_string)
print(loan_data_string)

['issue_d' 'loan_status' 'term' 'sub_grade' 'verification_status'
 'addr_state']
[['5' '1' '36' '13' '1' 'CA']
 ['0' '1' '36' '5' '1' 'NY']
 ['9' '1' '36' '10' '1' 'PA']
 ...
 ['6' '1' '36' '5' '1' 'CA']
 ['4' '1' '36' '17' '1' 'OH']
 ['12' '1' '36' '4' '0' 'IL']]


### State Address
for column State Address I renamed to be more understandable. The values in that columns is the abbreviations of all the states in United States plus some missing values so<br>1- the first thing to do is to replace the missing values with 0<br> when examining the number of records for each state I found out that every state separately does not give any insights as the the number of records are very low so <br>2- it would be better to divide them into 4 catigories (states_west, states_south, states_midwest, states_east)

In [552]:
loan_head_string[-1]='state_address'

In [553]:
# 1- the first thing to do is to replace the missing values with 0
loan_data_string[:,5] = np.where(loan_data_string[:,5] == '', 
                                  0, 
                                  loan_data_string[:,5])

In [554]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

In [555]:
# 2- it would be better to divide them into 4 catigories (states_west, states_south, states_midwest, states_east)

loan_data_string[:,-1] = np.where(np.isin(loan_data_string[:,-1], states_west), 1, loan_data_string[:,-1])
loan_data_string[:,-1] = np.where(np.isin(loan_data_string[:,-1], states_south), 2, loan_data_string[:,-1])
loan_data_string[:,-1] = np.where(np.isin(loan_data_string[:,-1], states_midwest), 3, loan_data_string[:,-1])
loan_data_string[:,-1] = np.where(np.isin(loan_data_string[:,-1], states_east), 4, loan_data_string[:,-1])

np.unique(loan_data_string[:,-1])

array(['0', '1', '2', '3', '4'], dtype='<U69')

### checking

In [556]:
loan_data_string

array([['5', '1', '36', '13', '1', '1'],
       ['0', '1', '36', '5', '1', '4'],
       ['9', '1', '36', '10', '1', '4'],
       ...,
       ['6', '1', '36', '5', '1', '1'],
       ['4', '1', '36', '17', '1', '3'],
       ['12', '1', '36', '4', '0', '3']], dtype='<U69')

In [557]:
counter=0
for i in loan_data_string.flatten():
    if i=='':
        counter+=1
counter

0

In [558]:
loan_data_string=loan_data_string.astype(int)
loan_data_string

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

<hr style="border:5px solid #108999"> </hr> 


# Numeric data processing

Now its time to process the numeric Data which is the cloumns ['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment',
'total_pymnt']<br>through these following steps we can alter these columns so that we get an array with no missing values and ready for processing <br>1- replace missing values with vallues of the worest case scenario
   

In [559]:
loan_head_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment',
       'total_pymnt'], dtype='<U11')

### loan ID

from this line we can see that the first column 'ID' doesnt have any missing values as expected.

In [560]:
np.isnan(loan_data_numeric[:,0]).sum()

0

### Loan Amount, Installments, Total Payments 
PS: the worest case scenario is to replace the messing values for Loaned Amount, Interest Rate, Total Payment and Installment with the maximum of these columns and the minimum of funded amount

In [561]:
max=np.nanmax(loan_data_numeric[:,(1,3,4,5)], axis=0)
np.nanmean(loan_data_numeric[:,[1,3,4,5]],axis=0)

array([15273.46315789,    16.61729479,   440.92217918,  3143.85094094])

In [562]:
counter=0
for i in [1,3,4,5]:
    loan_data_numeric[:,i]= np.where(np.isnan(loan_data_numeric[:,i]),
                               max[counter],
                               loan_data_numeric[:,i])
    counter+=1
np.nanmean(loan_data_numeric[:,[1,3,4,5]],axis=0)

array([16259.79      ,    24.045867  ,   487.617775  ,  5082.33939389])

In [563]:
# to display the interest rate as a decimal value 
loan_data_numeric[:,3]=loan_data_numeric[:,3]/100
loan_data_numeric[:,3]

array([0.1333, 0.2899, 0.2899, ..., 0.2899, 0.1655, 0.2899])

### Funded Rate

In [564]:
np.isnan(loan_data_numeric[:,2]).sum()

500

In [565]:
min=np.nanmin(loan_data_numeric[:,2])
np.nanmean(loan_data_numeric[:,2])

15311.042105263157

In [566]:
loan_data_numeric[:,2]= np.where(np.isnan(loan_data_numeric[:,2]),
                               min,
                               loan_data_numeric[:,2])
np.nanmean(loan_data_numeric[:,2])

14595.49

### The Exchange Rate
here I used another dataset for the exchange (EUR - USD) it consists of 12 rows for 12 months of the year 2015 in which this loan data is collected.<br>so we want to create another column that contains the exchange rates for each loan and attach it to loan_data_neumeric array

In [567]:
EUR_USD = np.genfromtxt("D:\\Data Processing with Numpy\\EUR-USD.csv", delimiter = ',', autostrip = True, skip_header = 1, usecols = 3)
EUR_USD

array([1.1287955 , 1.12053609, 1.08302462, 1.11143219, 1.09603453,
       1.12229669, 1.0939244 , 1.13400543, 1.12559378, 1.10089719,
       1.05830181, 1.09339809])

In [568]:
loan_data_string[0,0]

5

In [569]:

exchange_rate = loan_data_string[:,0]

for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)    

exchange_rate = np.where(exchange_rate == 0,
                         np.mean(EUR_USD),
                         exchange_rate)

exchange_rate

array([1.09603453, 1.10568669, 1.12559378, ..., 1.12229669, 1.11143219,
       1.09339809])

In [570]:
exchange_rate.shape

(10000,)

In [571]:
exchange_rate=np.reshape(exchange_rate,(10000,1))


In [572]:
loan_data_numeric=np.hstack((loan_data_numeric,exchange_rate))
loan_data_numeric[:,-1]

array([1.09603453, 1.10568669, 1.12559378, ..., 1.12229669, 1.11143219,
       1.09339809])

In [573]:
loan_head_numeric=np.hstack((loan_head_numeric, np.array(['Exchange_Rate'])))

In [574]:
loan_head_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment',
       'total_pymnt', 'Exchange_Rate'], dtype='<U13')

### from USD to EUR
here we want to change the numerical values in columns (Loan amount, Installments, total payments, fund Rates) from USD to EUR using the newly attached column (exchange rate )

In [575]:
counter=0
EUR_amount=np.zeros((loan_data_numeric.shape[0],4))

for i in [1,2,4,5]:
    EUR_amount[:,counter]=loan_data_numeric[:,i]/loan_data_numeric[:,6]
    counter+=1

EUR_amount


array([[31933.30058807, 31933.30058807,  1081.04258671,  8624.69180363],
       [27132.45999286, 27132.45999286,   848.85709918,  4232.39243429],
       [13326.29963573, 13326.29963573,   439.64350918,  1750.03632076],
       ...,
       [ 8910.29981708,  8910.29981708,  1223.35743399,  1947.47076922],
       [31490.89990968,  8997.39997419,   318.77788109,  2878.62814774],
       [ 9145.79973502,  9145.79973502,   283.49235439,   276.111694  ]])

In [576]:
loan_data_numeric=np.hstack((loan_data_numeric, EUR_amount))
loan_data_numeric.shape

(10000, 11)

In [577]:
# adding EUR columns name to the header

loan_head_numeric=np.hstack((loan_head_numeric,np.array([i+'_EUR' for i in loan_head_numeric[[1,2,4,5]]])))
loan_head_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment',
       'total_pymnt', 'Exchange_Rate', 'loan_amnt_EUR', 'funded_amnt_EUR',
       'installment_EUR', 'total_pymnt_EUR'], dtype='<U15')

In [578]:
# adding _USD to the columns name to specify their currency

for i in [1,2,4,5]:
    loan_head_numeric[i]+='_USD'
loan_head_numeric

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate',
       'installment_USD', 'total_pymnt_USD', 'Exchange_Rate',
       'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR',
       'total_pymnt_EUR'], dtype='<U15')

In [579]:
# rearranging the columns
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]
loan_data_numeric=loan_data_numeric[:,columns_index_order]
loan_head_numeric=loan_head_numeric[columns_index_order]
loan_head_numeric

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD',
       'funded_amnt_EUR', 'int_rate', 'installment_USD',
       'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR',
       'Exchange_Rate'], dtype='<U15')

<hr style="border:5px solid #108999"> </hr> 

# Create the complete Dataset

what I will do here :<br>1- concatenate the headers (string, numeric)<br>2-concatenate the data columns<br>3- sort the records ascending depending on the id

In [580]:
# 1- concatenate the headers (string, numeric)
loan_header=np.hstack((loan_head_numeric, loan_head_string))
loan_header

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD',
       'funded_amnt_EUR', 'int_rate', 'installment_USD',
       'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR',
       'Exchange_Rate', 'issue_d', 'loan_status', 'term', 'sub_grade',
       'verification_status', 'state_address'], dtype='<U19')

In [581]:
# 2-concatenate the data columns

loan_data=np.hstack((loan_data_numeric, loan_data_string))
loan_data

array([[48010226.        ,    35000.        ,    31933.30058807, ...,
              13.        ,        1.        ,        1.        ],
       [57693261.        ,    30000.        ,    27132.45999286, ...,
               5.        ,        1.        ,        4.        ],
       [59432726.        ,    15000.        ,    13326.29963573, ...,
              10.        ,        1.        ,        4.        ],
       ...,
       [50415990.        ,    10000.        ,     8910.29981708, ...,
               5.        ,        1.        ,        1.        ],
       [46154151.        ,    35000.        ,    31490.89990968, ...,
              17.        ,        1.        ,        3.        ],
       [66055249.        ,    10000.        ,     9145.79973502, ...,
               4.        ,        0.        ,        3.        ]])

In [582]:
# 3- sort the records ascending depending on the id

loan_data=loan_data[np.argsort(loan_data[:,0]),:]
loan_data

array([[  373332.        ,     9950.        ,     9038.08281434, ...,
              21.        ,        0.        ,        1.        ],
       [  575239.        ,    12000.        ,    10900.2003791 , ...,
              25.        ,        1.        ,        2.        ],
       [  707689.        ,    10000.        ,     8924.29980495, ...,
              13.        ,        1.        ,        0.        ],
       ...,
       [68614880.        ,     5600.        ,     5121.64785161, ...,
               8.        ,        1.        ,        1.        ],
       [68615915.        ,     4000.        ,     3658.31989401, ...,
              10.        ,        1.        ,        2.        ],
       [68616519.        ,    21600.        ,    19754.92742765, ...,
               3.        ,        0.        ,        2.        ]])

In [583]:
loan_header=loan_header.reshape((1,17))
loan_header.shape

(1, 17)

In [584]:
loan_data_Final=np.vstack((loan_header, loan_data))
loan_data_Final

array([['id', 'loan_amnt_USD', 'loan_amnt_EUR', ..., 'sub_grade',
        'verification_status', 'state_address'],
       ['373332.0', '9950.0', '9038.082814338286', ..., '21.0', '0.0',
        '1.0'],
       ['575239.0', '12000.0', '10900.20037910145', ..., '25.0', '1.0',
        '2.0'],
       ...,
       ['68614880.0', '5600.0', '5121.647851612413', ..., '8.0', '1.0',
        '1.0'],
       ['68615915.0', '4000.0', '3658.319894008867', ..., '10.0', '1.0',
        '2.0'],
       ['68616519.0', '21600.0', '19754.927427647883', ..., '3.0', '0.0',
        '2.0']], dtype='<U32')

In [585]:
np.savetxt("D:\\Data Processing with Numpy\\loan-data-preprocessed.csv", 
           loan_data_Final, 
           fmt = '%s',
           delimiter = ',')